In [1]:
import json

In [2]:
dataset = 'aishell'
recog_set = ['train', 'dev', 'test']
nbest = 50

In [ ]:
for r in recog_set:
    print(r)
    json_name = f'./data/{dataset}_{r}/{nbest}_best/data.json'
    with open(json_name, 'r') as f,\
         open(f'./data/{dataset}_{r}/{nbest}_best/hyp.trn', 'w') as hyp,\
        open(f'./data/{dataset}_{r}/{nbest}_best/ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
            assert(len(j['utts'][k]['output']) == nbest)
            for i, h in enumerate(j['utts'][k]['output']):
                # print(j['utts'][k]['output'])
                seq = [t for t in h['rec_token'].split()]
                hyp_seq = " ".join(seq[:-1])
                hyp.write( f'{hyp_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )
                # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')

                ref_seq = h["token"]
                ref.write( f'{ref_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )

上面code跑完 執行espnet的
```sclite -r ./ref.trn -h  ./hyp.trn -i rm -o all stdout >  ./result.txt```

In [5]:
recog_set = ['dev', 'test']

In [6]:

for task in recog_set:
    nbest_err = []
    print(task)
    with open(f'./data/aishell_{task}/{nbest}_best/result.txt', 'r') as f:
        temp_err = []
        for i, line in enumerate(f):
            if ('Score' in line):
                scores = line.split()[-4:]
                temp_err.append(scores)
                if (len(temp_err) == nbest):
                    nbest_err.append(temp_err)
                    temp_err = []
        names = []
        nbest_token = []
        nbest_score = []
        ref_text = []
        with open(f'./data/aishell_{task}/{nbest}_best/data.json', 'r') as f:
            j = json.load(f)
            for k in j['utts'].keys():
                token = []
                score = []
                for i, h in enumerate(j['utts'][k]['output']):
                    token.append(h["rec_text"])
                    score.append(h['score'])
                nbest_token.append(token)
                nbest_score.append(score)
                ref_text.append(j['utts'][k]['output'][0]['text'])
                names.append(k)

        print(len(ref_text))
        dataset = []
        for name, token, score, err, ref in zip(names, nbest_token, nbest_score, nbest_err, ref_text):
            temp_dict = dict()
            temp_dict['name'] = name
            temp_dict['token'] = ['[CLS]' + t[:-5] + '[SEP]' for  t  in token]
            temp_dict['score'] = score
            temp_dict['err'] =  [[int(s) for s in sc] for sc in err]
            temp_dict['ref'] = '[CLS]' + ref + '[SEP]'
            dataset.append(temp_dict)
        print(len(dataset))
    with open(f"./data/aishell_{task}/{nbest}_best/dataset.json", 'w') as f:
        json.dump(dataset, f, ensure_ascii=False, indent = 4)

dev
14326
14326
test
7176
7176


## Result trn

In [30]:
dataset = 'aishell'
name = "vote"
recog_set = ['dev', 'test']
nBest = 50
training = 'MWER'

# BART

In [ ]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/50_best/bart_token/rescore_data.json'
    with open(json_name, 'r') as f, \
            open(f'./data/{dataset}_{r}/10_best/bart_token/hyp.trn', 'w') as hyp, \
            open(f'./data/{dataset}_{r}/10_best/bart_token/ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['text_token']
            ref.write( f'{ref_seq} ({k})\n' )

# RescoreBert

In [31]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/50_best/rescore/{training}_rescore_data.json'
    with open(json_name, 'r') as f, \
        open(f'./data/{dataset}_{r}/50_best/rescore/{nBest}best_hyp.trn', 'w') as hyp, \
        open(f'./data/{dataset}_{r}/50_best/rescore/{nBest}best_ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['text_token']
            ref.write( f'{ref_seq} ({k})\n' )

In [ ]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/nBestFusionNet/rescore_data.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/nBestFusionNet/hyp_rescore.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/nBestFusionNet/ref_rescore.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['text_token']
            ref.write( f'{ref_seq} ({k})\n' )

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
import json
import torch
# for r in recog_set:
best_weight = 0
best_cer = 100
with open(f'./data/aishell_dev/token_pll.json') as f:
    data = json.load(f)
    
    for n in range(101):
        weight = n * 0.01
        c = 0
        s = 0
        de = 0
        i = 0
        for d in data:
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            c += cer[max_index][0]
            s += cer[max_index][1]
            de += cer[max_index][2]
            i += cer[max_index][3]
 
        cer = (s + de + i) / (c + s + de)

        if (cer < best_cer):
            best_cer = cer
            best_weight = weight

print(f'best weight:{best_weight} with cer:{best_cer}')

for r in recog_set:
    with open(f'./data/aishell_{r}/token_pll.json', 'r') as f, \
         open(f'./data/aishell_{r}/rescore/hyp_mlm.trn', 'w') as h,\
         open(f'./data/aishell_{r}/rescore/ref_mlm.trn', 'w') as g:
        data = json.load(f)
        for n, d in enumerate(data):
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
            ref = d['ref'][5:-5]
            token = d['token']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            best_hyp = token[max_index]
            sep = best_hyp.index(102)
            hyp_seq = tokenizer.convert_ids_to_tokens(best_hyp[1:sep])

            h.write( f'{" ".join(hyp_seq)} ({r}_{n})\n' )
            g.write( f'{" ".join(list(ref))} ({r}_{n})\n')
